# ChatGPT를 활용한 자연어처리 자동화

대규모 언어 모델 (LLM)이 계속 커져나감에 따라, 웬만한 기업들조차 최첨단 언어 모델을 처음부터

학습하기도 무리인 동시에, 그 모델을 로컬에서 실행시킬 수 있는 환경이 안되는 경우가 태반입니다.

- ChatGPT의 경우 서비스만을 위해 사용되는 GPU가 (Nvidia A100) 2만개 이상으로 추산됨
- Meta와 X는 다음 세대 LLM을 위하여 10만개 이상의 GPU를 학습에 사용중에 있음


우리가 만약 최첨단 성능과 안정성을 바란다면, 비용을 지불하고 ChatGPT나 Claude이 제공하는 API를 통해

쿼리를 주고 받으면서 이에 대한 처리를 요청할 수 있습니다.

이 예제코드는 ChatGPT를 활용하여 과거의 뉴스 기사들을 일별로 요약 및 번역을 해봅니다.

# Requirements

Python 패키지
- pandas
- transformers
- langchain
- langchain-community
- langchain-openai

In [2]:
!pip install pandas transformers langchain langchain-community langchain-openai python-dotenv

# OpenAI API 발급받기 / 요금 충전

OpenAI를 사용하기 위해서는 API를 발급하고 미리 크레딧을 충전해놓아야 합니다.

(프로젝트를 생성)

1. OpenAI 플랫폼 로그인/회원가입

https://platform.openai.com/

2. 프로필로 이동

[그림]

3. 좌측 API keys로 이동

[그림]

4. "Create new secret key" 클릭

[그림]

5. 이름을 정하거나 해서 Create secret key 클릭

[그림]

6. 복사 후 어디다가 저장하기

[그림]


<h3>크레딧 충전하기</h3>


1. 프로필 => 좌측 Billing으로 이동, Add payment detail 선택

[그림]

2. 해외결제 되는 카드 결제수단 입력

3. Configure payment에서 상단에 최초 금액 입력, 하단에 자동 충전 설정 (꺼도 됨)

[그림]



# ChatGPT 불러오기

Langchain 라이브러리는 OpenAI API와 소통할 수 있는 간단한 클래스를 제공해줍니다.

다음은 간단하게 OpenAI의 `gpt-3.5-turbo`를 이용해보는 코드입니다

In [ ]:
import os

import dotenv
from langchain_openai import ChatOpenAI

# .env 파일을 생성하고 그 안에 
# OPENAI_API_KEY=sk-proj-어쩌구저쩌구API키
# 를 입력하거나 아래 os.environ 선언하는 것에 입력하세요.
dotenv.load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "" # 여기에 발급받은 API 키를 입력하세요.


# 비용이 발생합니다.
llm = ChatOpenAI(
    temperature = 0.0,
    max_tokens=2048,
    model_name = "gpt-4o-mini"
)

question = "대한민국을 한 문단 내로 간략히 설명해주세요."

print(f"{llm.invoke(question).content}")

대한민국은 동아시아에 위치한 한반도의 남쪽에 있는 국가로, 북쪽으로는 북한과 접경하고 있으며, 서쪽은 황해, 동쪽은 동해, 남쪽은 대한해협에 접해 있습니다. 1948년에 정부가 수립된 이후 민주주의 체제를 갖추고 있으며, 경제적으로는 세계에서 가장 빠르게 성장한 국가 중 하나로, IT, 자동차, 조선업 등 다양한 산업에서 강력한 경쟁력을 보이고 있습니다. 서울은 수도이자 문화, 경제, 정치의 중심지로, 전통과 현대가 조화를 이루는 도시입니다. 한국의 독특한 문화와 역사, K-팝, 한식 등은 세계적으로 큰 영향을 미치고 있습니다.


# 뉴스 데이터 불러오기

우리는 과거 러시아-우크라이나 전쟁 뉴스를 가지고
- 하루단위로 있었던 해외 뉴스 기사 내 사건들을 요약하고
- 이를 한글로 번역해서 보여주도록

ChatGPT에게 지시를 해보는 예제를 만들어 볼 것입니다.

In [9]:
import pandas as pd

news_df = pd.read_csv("data/Guardians_Russia_Ukraine.csv")
news_df.head(5)

,published,headlines,articles
0,2022-08-01T16:23:25Z,Sanctions against Russia are not backfiring | ...,Simon Jenkins (The rouble is soaring and Putin...
1,2022-07-26T07:54:56Z,Can Ukrainian forces recapture Kherson from Ru...,"In the first phase of the war in Ukraine, the ..."
2,2022-08-05T15:00:03Z,Nightlands review – talking through what’s bec...,Who exactly is the enemy currently laying wast...
3,2022-08-02T16:28:09Z,Russia claims US ‘directly involved’ in Ukrain...,The role of American intelligence in the war i...
4,2022-07-27T12:40:40Z,Is Russia killing off the International Space ...,"The International Space Station, which circles..."


데이터를 시간순으로 정렬합니다.

우리는 각 일별로 데이터를 보고 싶으므로, 년-월-일 까지의 레코드를 새롭게 만들어 볼 것입니다.

In [16]:
news_df = news_df.sort_values(by="published")

news_df["date"] = news_df["published"].apply(lambda x: x[:10])

# 기사의 일별 날짜 종류 (중복 제거)
dates = news_df["date"].unique()

In [ ]:
# 살펴볼 날짜들
dates

array(['2022-02-24', '2022-02-28', '2022-03-10', '2022-03-21',
       '2022-03-28', '2022-04-04', '2022-04-11', '2022-04-12',
       '2022-04-14', '2022-04-19', '2022-04-22', '2022-04-25',
       '2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29',
       '2022-04-30', '2022-05-02', '2022-05-04', '2022-05-05',
       '2022-05-06', '2022-05-08', '2022-05-09', '2022-05-10',
       '2022-05-13', '2022-05-19', '2022-05-20', '2022-05-21',
       '2022-05-22', '2022-05-23', '2022-05-26', '2022-05-27',
       '2022-05-28', '2022-05-31', '2022-06-01', '2022-06-04',
       '2022-06-05', '2022-06-06', '2022-06-07', '2022-06-08',
       '2022-06-10', '2022-06-11', '2022-06-12', '2022-06-13',
       '2022-06-14', '2022-06-15', '2022-06-16', '2022-06-17',
       '2022-06-18', '2022-06-19', '2022-06-20', '2022-06-21',
       '2022-06-22', '2022-06-23', '2022-06-24', '2022-06-25',
       '2022-06-26', '2022-06-27', '2022-06-28', '2022-06-29',
       '2022-07-01', '2022-07-02', '2022-07-03', '2022-

# 입력 전처리하기

우리는 GPT에게 그 날에 있었던 기사들을 한번에 보내어 그 하루 내 있었던 이슈들을 요약하여 반환하게 하고 싶습니다.

먼저 일 단위로 하루에 있었던 모든 기사들을 일정한 형식으로 묶어내도록 전처리하겠습니다.

In [ ]:
# 입력받은 모든 기사를 하나의 문자열로 반환합니다.
def preprocessing(df):
    articles_str = ""
    for idx in df.index:
        articles_str += f"""
Title: {df.loc[idx, "headlines"]}
Body:
{df.loc[idx, "articles"]}\n
"""
    return articles_str

# 기사 몇개로 전처리를 테스트해봅시다.

tgt_date = "2022-02-24"
test_news_df = news_df[news_df["date"]==tgt_date]
print(preprocessing(test_news_df))


Title: Thursday briefing: Russia launches attack on Ukraine
Body:
Top story: Kremlin invokes ‘demilitarisation’ as excuse

Hello – Warren Murray introducing the news on a day we feared would come.

Russian forces have attacked Ukraine on the orders of Vladimir Putin, who announced what he called a “special military operation” at dawn. Explosions and airstrikes have been heard this morning near major Ukrainian cities, including the capital, Kyiv, and air raid sirens went off. Our correspondent Luke Harding reported earlier: “At least seven ‘powerful’ airstrikes on Vasilkovsky airport outside Kyiv, where Ukraine’s military fighter bombers are kept, Kyiv command says.” The Ukraine foreign minister, Dmytro Kuleba, confirmed what he called the “full-scale invasion of Ukraine. Peaceful Ukrainian cities are under strikes.”

00:54 Ukraine: Russian missile strikes reported in Kharkiv and Kyiv – video

In a bid to justify the invasion, Putin branded Ukraine an “anti-Russia … created on our hist

# 파이프라인 설계 및 자동화 결과 얻기

앞에서 전처리하는 함수를 이용하여, 기사 내용을 넣어주면 그 날에 있었던 이슈들을 요약해서 한글로 알려주도록 GPT에게 질의하는 파이프라인을 설계해보겠습니다.

지시어를 잘 주어야 좋은 결과를 얻을 수 있습니다. 일단 간단한 지시어부터 시작해봅시다.

In [34]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 지시어를 잘 주어야 합니다. 간단한 것으로 시작해봅시다.
template = """
다음은 우크라이나-러시아 전쟁 뉴스 기사들입니다. 그 기사들을 보고, 있었던 일들을 요약하고 한글로 번역해주세요.
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

In [35]:
tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

### 요약 및 번역

**요약:**
2022년 2월 24일, 러시아는 우크라이나에 대한 대규모 공격을 시작했다. 블라디미르 푸틴 대통령은 "특별 군사 작전"을 선언하며 우크라이나를 "비무장화"하고 "탈나치화"하겠다고 주장했다. 공격은 키이우, 하르키우, 그리고 헤르손을 포함한 주요 도시를 목표로 하였으며, 공습과 미사일 공격이 이어졌다. 우크라이나 외무장관은 이를 "전면 침공"으로 간주하며, 평화로운 도시들이 공격받고 있다고 밝혔다. 국제 사회는 러시아의 공격에 대해 강력히 반발하고 있으며, 미국과 영국은 우크라이나를 지원할 것이라고 밝혔다. 푸틴은 서방의 군사 개입에 대해 강력한 경고를 하였고, 우크라이나의 동부 지역을 러시아에 통합할 의도를 내비쳤다. 전투는 체르노빌과 같은 전략적 장소에서도 발생하고 있으며, 우크라이나 군은 저항하고 있다. 현재까지의 사망자는 57명으로 보고되었고, 전투는 계속되고 있다.

**번역:**
2022년 2월 24일, 러시아는 우크라이나에 대한 대규모 공격을 시작했습니다. 블라디미르 푸틴 대통령은 "특별 군사 작전"을 선언하며 우크라이나를 "비무장화"하고 "탈나치화"하겠다고 주장했습니다. 공격은 키이우, 하르키우, 그리고 헤르손을 포함한 주요 도시를 목표로 하였으며, 공습과 미사일 공격이 이어졌습니다. 우크라이나 외무장관은 이를 "전면 침공"으로 간주하며, 평화로운 도시들이 공격받고 있다고 밝혔습니다. 국제 사회는 러시아의 공격에 대해 강력히 반발하고 있으며, 미국과 영국은 우크라이나를 지원할 것이라고 밝혔습니다. 푸틴은 서방의 군사 개입에 대해 강력한 경고를 하였고, 우크라이나의 동부 지역을 러시아에 통합할 의도를 내비쳤습니다. 전투는 체르노빌과 같은 전략적 장소에서도 발생하고 있으며, 우크라이나 군은 저항하고 있습니다. 현재까지의 사망자는 57명으로 보고되었고, 전투는 계속되고 있습니다.


출력 형식이 마음에 들지 않을 수 있습니다. 이럴 땐 지시어로 출력을 구조화하도록 유도할 수 있습니다.

In [44]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 이때 사건 단위로 요약하여 다음처럼 나열해주세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

출력 형식
```
**요약:**
- 사건 1
- 사건 2
- 사건 3
- 사건 4
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

In [45]:
tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

```
**요약:**
- 러시아군이 우크라이나를 공격하라는 푸틴의 명령에 따라 전면적인 군사 작전을 시작했다. 주요 도시인 키이우와 하르키우에서 폭발과 공습이 발생했다.
- 우크라이나 외무장관 드미트로 쿨레바는 이를 "전면 침공"으로 확인하며, 평화로운 도시들이 공격받고 있다고 밝혔다.
- 푸틴은 우크라이나를 "반러시아"로 묘사하며, "비무장화 및 탈나치화"를 목표로 한다고 주장했다. 이는 크렘린의 허위 선전과 일치하는 발언이다.
- 우크라이나 대통령 볼로디미르 젤렌스키는 러시아 국민에게 "이성의 목소리를 들어달라"며 직접 호소했다. 그는 "우리의 땅과 자유를 빼앗으려는 자가 있다면 우리는 스스로 방어할 것"이라고 경고했다.
- 러시아군은 키이우 근처의 호스톰엘 공항을 공격하며 공중 기지를 확보하려고 시도하고 있으며, 우크라이나는 이에 맞서 싸우고 있다.
- 체르노빌 원자력 발전소가 러시아군의 통제 하에 들어갔으며, 이로 인해 방사능 위험이 우려되고 있다.
- 하르키우와 남부의 헤르손에서 격렬한 전투가 벌어지고 있으며, 러시아군은 크림 반도에서 우크라이나로 진입하고 있다.
- 초기 공습에서 우크라이나 측의 사망자는 57명, 부상자는 169명으로 보고되었으며, 군인과 민간인 사상자가 발생하고 있다.
```


잘 나오는 것으로 보이니, 모든 날짜에 대해서 자동화를 해보겠습니다.

In [ ]:
result = ""

for tgt_date in news_df["date"].unique():
    print("날짜: ", tgt_date)
    articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
    answer = chain.invoke(input={"articles":articles_str})
    print(answer)
    result += f"날짜: {tgt_date}\n{answer}\n"

# 파일로 저장해봅니다.
with open("우크라이나-러시아 전쟁_요약본.txt","w") as file:
    file.write(result)

날짜:  2022-02-24
```
**요약:**
- 러시아군이 우크라이나를 공격하라는 푸틴의 명령에 따라 전면적인 군사 작전을 시작했다. 주요 도시인 키이우와 하르키우에서 폭발과 공습이 발생했다.
- 우크라이나 외무장관 드미트로 쿨레바는 이를 "전면 침공"으로 확인하며, 평화로운 도시들이 공격받고 있다고 밝혔다.
- 푸틴은 우크라이나를 "반러시아"로 묘사하며, "비무장화 및 탈나치화"를 목표로 한다고 주장했다. 이는 크렘린의 허위 선전과 일치하는 발언이다.
- 우크라이나 대통령 볼로디미르 젤렌스키는 러시아 국민에게 "이성의 목소리를 들어달라"며 직접 호소했으며, 우크라이나는 자신을 방어할 것이라고 경고했다.
- 러시아의 공격은 북쪽, 동쪽, 남쪽에서 동시에 이루어졌으며, 키이우 근처의 호스톰엘 공항을 포함한 전략적 공항을 차지하기 위한 격렬한 전투가 벌어졌다.
- 체르노빌 원자력 발전소가 러시아군의 통제 하에 들어갔으며, 이로 인해 방사능 위험이 우려되고 있다.
- 하르키우와 남부의 헤르손에서 러시아군의 공격이 집중되고 있으며, 우크라이나 군은 저항하고 있다.
- 초기 공습에서 우크라이나 측의 사망자는 57명, 부상자는 169명으로 보고되었으며, 민간인 사상자도 발생했다.
```
날짜:  2022-02-28
```
**요약:**
- 스위스가 러시아에 대한 EU 제재를 전면적으로 채택하기로 결정하였으며, 이는 러시아 경제에 추가적인 압박을 가할 것으로 보인다.
- 스위스 정부는 블라디미르 푸틴과 가까운 5명의 올리가르흐의 입국을 금지하고, 러시아에서의 항공편을 차단하기로 했다.
- 핀란드 정부는 우크라이나에 무기를 공급하기로 한 "역사적인" 결정을 발표하였다.
- 우크라이나 대통령 볼로디미르 젤렌스키가 스위스 대통령에게 우크라이나와 러시아 간의 중재자로서 역할을 요청하였다.
```
날짜:  2022-03-10
```
**요약:**
- 서방 관리들은 블라디미르 푸틴이 우크라이나에 화학 무기를 사용할 가능성에 대해 심각한 우려를 표명했다. 미국은 러시아의 생물학적

# 이것저것 바꾸어보기

## 1. 서술식 문장 => 개조식 문장으로


In [48]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 이때 사건 단위로 요약하여 다음처럼 나열해주세요.
- 서술식이 아닌 개조식으로 적어주세요.
  - 개조식 예시: 러시아는 미국과의 포로 교환을 논의할 준비가 되어있다고 밝힘
- 영어로 적지 말고 바로 한글로 적어주세요.

출력 형식
```
**요약:**
- 사건 1
- 사건 2
- 사건 3
- 사건 4
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

```
**요약:**
- 러시아군이 우크라이나를 공격하며 "특별 군사 작전"을 시작했다고 푸틴이 발표함.
- 우크라이나 외무장관이 "전면 침공"을 확인하며 평화로운 도시들이 공격받고 있다고 언급함.
- 푸틴이 우크라이나를 "반러시아"로 묘사하며 군사 작전의 정당성을 주장함.
- 조 바이든과 보리스 존슨이 러시아의 공격에 대한 국제적 비난을 표명하며 우크라이나를 지지함.
- 우크라이나 대통령 젤렌스키가 러시아 국민에게 "이성의 목소리"를 들을 것을 호소함.
- 러시아의 국방부 장관과 군 고위 관계자들이 EU 제재 대상이 될 것으로 예상됨.
- 우크라이나가 체르노빌 원자력 발전소의 통제권을 잃었다고 발표함.
- 우크라이나의 여러 도시에서 격렬한 전투가 발생하고 있으며, 특히 하르키우와 키예프에서 큰 피해가 보고됨.
- 우크라이나 보건부가 57명이 사망하고 169명이 부상당했다고 발표함.
- 러시아군이 우크라이나 국경 근처에 15만 명 이상의 전투 병력을 집결시킴.
```


## 2. 각 뉴스 기사를 따로 요약하고, 마지막에 하루 전체를 요약하도록 하기

In [52]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 기사 마다 각각 요약한 다음 마지막으로 보았던 모든 기사의 사건들을 모아 요약해주세요
- 일어난 사건 단위로 요약하여 나열해주세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

출력 형식
```
**요약:**
기사 1 제목: (기사 1의 제목입니다)
- (사건 1)
- (사건 2)
- (사건 3)

기사 2 제목: (기사 2의 제목입니다)
- (사건 1)
- (사건 2)

기사 3 제목: (기사 3의 제목입니다)
- (사건 1)
- (사건 2)
- (사건 3)
- (사건 4)

전체 요약:
- (사건 1)
- (사건 2)
- (사건 3)
- (사건 4)
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

```
**요약:**
기사 1 제목: 목요일 브리핑: 러시아, 우크라이나에 대한 공격 개시
- 블라디미르 푸틴의 명령으로 러시아군이 우크라이나를 공격하기 시작했다.
- 우크라이나 외무장관 드미트로 쿨레바는 "전면 침공"을 확인했다.
- 푸틴은 우크라이나를 "반러시아"로 묘사하며 침공의 정당성을 주장했다.
- 우크라이나 대통령 볼로디미르 젤렌스키는 러시아 국민에게 "이성의 목소리를 들어달라"고 호소했다.
- 국제 사회의 비난이 커지고 있으며, EU는 러시아 고위 인사들에 대한 제재를 예고했다.

기사 2 제목: 러시아를 막을 수 있는 사람이 있는가?
- 푸틴은 서방의 간섭에 대해 강력한 경고를 했다.
- 그는 우크라이나의 동부 지역을 러시아에 통합할 계획을 시사했다.
- 러시아 내 반대 목소리는 억압당하고 있으며, 푸틴은 전권을 행사하고 있다.
- 서방의 제재가 효과를 발휘할 가능성은 낮다고 평가되었다.

기사 3 제목: 우크라이나 전쟁: 러시아의 공격 위치
- 러시아의 공격은 오전 5시경 시작되었으며, 키이우, 하르키우, 헤르손을 목표로 삼았다.
- 우크라이나는 체르노빌 원자력 발전소의 통제권을 잃었다고 발표했다.
- 하르키우에서의 격렬한 전투가 보고되었으며, 러시아군이 키이우 근처의 공군 기지를 장악하려고 시도하고 있다.
- 초기 공격으로 인해 우크라이나에서 57명이 사망하고 169명이 부상당했다.

전체 요약:
- 러시아군이 블라디미르 푸틴의 명령으로 우크라이나를 전면 침공했다.
- 우크라이나 외무장관은 이를 "전면 침공"으로 확인했으며, 젤렌스키는 러시아 국민에게 호소했다.
- 푸틴은 서방의 간섭에 대해 강력한 경고를 하며, 우크라이나 동부 지역의 통합을 시사했다.
- 러시아군은 키이우, 하르키우, 헤르손을 목표로 공격을 감행하고 있으며, 체르노빌 원자력 발전소의 통제권을 잃었다.
- 초기 공격으로 인해 우크라이나에서 다수의 사상자가 발생했다.
```


In [ ]:
# 제목 검증해보기
for h in news_df[news_df["date"]==tgt_date]["headlines"]: print(h)

Thursday briefing: Russia launches attack on Ukraine
Can anyone in Russia stop Putin now? | Angus Roxburgh
War in Ukraine: where has Russia attacked?


In [58]:
# 날짜를 바꾸어서 한번 더
prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-08-01"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

```
**요약:**
기사 1 제목: 우크라이나 항구에서 러시아 봉쇄 이후 처음으로 곡물 선박 출항
- 우크라이나 오데사 항구에서 곡물 선박이 러시아의 침공 이후 처음으로 출항했다.
- 시에라리온 국적의 선박 '라조니'가 26,000톤의 옥수수를 실고 레바논으로 향했다.
- 우크라이나 인프라 장관은 이 출항이 세계 기아를 예방하는 데 기여한다고 밝혔다.
- 러시아는 이 출항 소식을 긍정적으로 평가했으며, 추가 선박 출항이 예정되어 있다고 전했다.
- 전투는 계속되고 있으며, 미국과 독일의 장거리 로켓 시스템이 우크라이나에 도착했다.
- 미콜라이우 시장은 도시의 포격이 가장 심각했다고 전하며, 사망자 중에는 우크라이나의 부유한 농업 재벌도 포함되었다.
- 도네츠크 지역에서 전투로 인해 3명이 사망하고 16명이 부상당했다.

기사 2 제목: 러시아-우크라이나 전쟁 최신 소식: 침공 159일째
- 미콜라이우 시장은 도시의 의료 시설에 대한 공격을 "러시아 군대의 냉혹한 테러"라고 비난했다.
- 공격으로 인해 응급 병원이 환자를 받을 수 없게 되었다고 전했다.

기사 3 제목: 키어 스타머와 피어스 모건, 러시아 입국 금지 명단에 포함
- 러시아가 39명의 영국 정치인, 사업가, 언론인을 입국 금지 명단에 추가했다.
- 이 명단에는 노동당 리더 키어 스타머와 전 총리 데이비드 카메론이 포함되었다.
- 러시아 외무부는 이 조치를 영국의 적대적 행동에 대한 반응으로 설명했다.
- 러시아는 독립 언론에 대한 단속을 강화하고 있으며, 여러 외국 뉴스 사이트에 대한 접근을 차단했다.

기사 4 제목: 제재가 러시아에 역효과를 주지 않는다 | 편지
- 제재가 러시아의 경제에 미치는 영향에 대한 논의가 이어졌다.
- 일부는 제재가 러시아의 국제적 통합을 깨뜨리는 데 효과적이라고 주장했다.
- 우크라이나 전쟁으로 인해 제재가 러시아에 대한 연대감을 나타내는 방법으로 여겨지고 있다.

전체 요약:
- 우크라이나 오데사 항구에서 곡물 선박이 출항하며 세계 기아 문제 해결에 기여하고 있

In [59]:
# 제목 검증해보기
for h in news_df[news_df["date"]==tgt_date]["headlines"]: print(h)

Grain ship leaves Ukraine port for first time since Russia blockade
Russia-Ukraine war latest: what we know on day 159 of the invasion
Keir Starmer and Piers Morgan among new list of Britons banned from Russia
Keir Starmer and Piers Morgan among new list of Britons banned from Russia
Sanctions against Russia are not backfiring | Letters


## 3. 전황과 인물들의 발언 모아보기


In [ ]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 다음 형식으로 있었던 일들을 요약하고 한글로 번역해주세요.
- 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해주세요.
- 전쟁 상황과 인물들의 발언을 따로 나누어서 제시하세요.
- 일어난 사건 단위로 요약하여 나열해주세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

출력 형식
```
전쟁 상황
- 사건 1
- 사건 2
- 사건 3
- 사건 4

관련 발언
- 발언 1
- 발언 2
- 발언 3
- 발언 4
- 발언 5
```

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

```
전쟁상황
- 러시아군이 우크라이나를 공격하기 시작했으며, 주요 도시인 키이우, 하르키우, 그리고 헤르손을 목표로 한 다각적인 침공이 진행되고 있다.
- 우크라이나는 체르노빌 원자력 발전소의 통제권을 잃었고, 이 지역에서의 전투가 격렬하게 벌어지고 있다.
- 하르키우에서 러시아군의 공격이 집중되고 있으며, 이 지역을 포위하려는 시도가 이루어지고 있다.
- 크림 반도에서 우크라이나로 진입한 러시아군이 헤르손에 도달했으며, 이곳에서도 격렬한 전투가 발생하고 있다.
- 우크라이나 보건부에 따르면, 전투로 인해 57명이 사망하고 169명이 부상당한 것으로 보고되었다.

관련 발언
- 우크라이나 외무장관 드미트로 쿨레바는 "우크라이나의 전면 침공이 시작되었다"고 확인했다.
- 블라디미르 푸틴은 우크라이나를 "반러시아"로 묘사하며, 이 전쟁의 목적이 "비무장화 및 탈나치화"라고 주장했다.
- 조 바이든 미국 대통령은 "전 세계의 기도가 우크라이나 국민과 함께하고 있다"고 말했다.
- 우크라이나 대통령 볼로디미르 젤렌스키는 "우리는 스스로를 방어할 것"이라고 강조하며 러시아 국민에게 이성을 들을 것을 호소했다.
- 러시아 국방부는 "74개의 군사 기반 시설을 파괴했다"고 주장했으나, 이는 확인되지 않았다.
```


## 4. 블로그 글 써보게 하기

In [64]:
template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 자유형식으로 2~3 문단 정도로 블로그 글을 써주세요.
- 정보를 알리려는 목적의 글이며, 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해야 합니다.
- 그 날에 올라온 기사들이다 라고 생각하고 블로그를 작성합니다.
- 길이는 최대 4문단 정도 되어야 합니다.
- 중대한 사건들은 강조해주세요.
- 독자들에게 친숙하게 구어체를 사용하세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

오늘 우크라이나-러시아 전쟁에 대한 뉴스는 매우 충격적이고 긴박한 상황을 전하고 있습니다. 러시아의 블라디미르 푸틴 대통령은 오늘 새벽 "특별 군사 작전"을 발표하며 우크라이나에 대한 전면 공격을 명령했습니다. 키이우를 포함한 주요 도시에서 폭발음과 공습 경고가 울렸고, 우크라이나 외무장관 드미트로 쿨레바는 이를 "전면 침공"으로 규정했습니다. 푸틴은 우크라이나를 "반러시아"로 묘사하며, 이 작전의 목적이 "비무장화 및 탈나치화"라고 주장했습니다. 이는 크렘린의 허위 선전과 일치하는 발언으로, 국제 사회의 강한 반발을 불러일으키고 있습니다.

우크라이나의 젤렌스키 대통령은 러시아 국민에게 직접 호소하며 "우리는 우리의 땅과 자유를 지킬 것"이라고 강조했습니다. 그는 러시아의 공격에 맞서 싸울 것임을 분명히 하며, 우크라이나 국민이 결코 물러서지 않을 것임을 알렸습니다. 현재 전투는 키이우 근처의 호스톰엘 공항과 하르키우, 그리고 남부의 헤르손에서 치열하게 벌어지고 있으며, 우크라이나 군은 강력한 저항을 보이고 있습니다. 특히, 체르노빌 원자력 발전소의 통제권이 러시아에 넘어갔다는 소식은 매우 우려스러운 상황입니다. 이곳에서의 전투가 방사능 유출로 이어질 수 있다는 경고도 나오고 있습니다.

국제 사회는 이 사태에 대해 강력히 반발하고 있으며, 미국과 영국을 포함한 여러 나라가 러시아에 대한 제재를 강화할 계획을 세우고 있습니다. 하지만 푸틴은 이러한 제재에도 아랑곳하지 않고, 전쟁을 지속할 의지를 보이고 있습니다. 현재 상황은 매우 긴박하며, 전 세계가 우크라이나의 상황을 주목하고 있습니다. 앞으로의 전개가 어떻게 될지, 많은 이들이 우려와 관심을 가지고 지켜보고 있습니다.


## 5. 더 좋은 모델 써보기 (4번 블로그글 쓰기를 좋은 모델로)

In [65]:
better_llm = ChatOpenAI(
    temperature = 0.0,
    max_tokens=2048,
    model_name = "gpt-4o"
)

template = """
우크라이나-러시아 전쟁 뉴스 기사들이 주어질 것입니다.
그 기사들을 보고, 자유형식으로 2~3 문단 정도로 블로그 글을 써주세요.
- 정보를 알리려는 목적의 글이며, 우크라이나-러시아 전쟁에 관련된 사건과 발언들을 요약해야 합니다.
- 그 날에 올라온 기사들이다 라고 생각하고 블로그를 작성합니다.
- 길이는 최대 4문단 정도 되어야 합니다.
- 중대한 사건들은 강조해주세요.
- 독자들에게 친숙하게 구어체를 사용하세요.
- 영어로 적지 말고 바로 한글로 적어주세요.

뉴스 기사
###
{articles}
###
"""

prompt = PromptTemplate.from_template(template=template)

chain = prompt | better_llm | StrOutputParser()

tgt_date = "2022-02-24"
articles_str = preprocessing(news_df[news_df["date"]==tgt_date])
answer = chain.invoke(input={"articles":articles_str})
print(answer)

오늘 아침, 우리가 두려워했던 일이 현실이 되었습니다. 러시아가 우크라이나를 공격했습니다. 블라디미르 푸틴 대통령은 "특별 군사 작전"을 명령하며, 우크라이나의 주요 도시들, 특히 수도 키이우와 하르키우, 그리고 남부의 헤르손을 목표로 삼았습니다. 이른 아침부터 폭발음과 공습 경보가 울리며, 우크라이나 전역이 긴장감에 휩싸였습니다. 우크라이나 외무장관 드미트로 쿨레바는 이를 "전면적인 침공"이라고 규정하며, 평화로운 도시들이 공격받고 있다고 전했습니다.

푸틴 대통령은 이번 침공을 정당화하기 위해 우크라이나를 "반(反)러시아"로 규정하며, "비무장화와 비나치화"를 목표로 한다고 주장했습니다. 이는 크렘린의 선전과 맥락을 같이하는 발언으로, 우크라이나 정부가 극우 세력에 의해 통제되고 있다는 잘못된 주장을 반복한 것입니다. 이에 대해 조 바이든 미국 대통령은 "러시아의 군사력에 의한 부당하고 정당하지 않은 공격"이라며 강력히 비난했습니다. 보리스 존슨 영국 총리 역시 "피와 파괴의 길을 선택한 푸틴 대통령"이라며, 영국과 동맹국들이 단호히 대응할 것임을 밝혔습니다.

국제 사회의 비난이 커지는 가운데, 러시아의 국방장관과 주요 인사들이 유럽연합의 제재 대상에 오를 것으로 보입니다. 그러나 러시아는 이미 2014년 크림반도 침공 이후 서방의 경제적 제재에 대비해 왔으며, 상당한 외환 보유고와 낮은 국가 부채 수준을 유지하고 있습니다. 이러한 경제적 방어막이 제재의 효과를 약화시킬 수 있다는 분석도 나오고 있습니다. 유럽은 러시아산 가스에 대한 의존도가 높아, 에너지 공급 차질에 대한 우려도 커지고 있습니다. 미국과 다른 국가들은 러시아가 가스 공급을 중단할 경우, 이를 보완할 준비가 되어 있다고 밝혔습니다.

우크라이나의 상황은 매우 심각합니다. 러시아군은 키이우 인근의 전략적 공군 기지를 점령하려 시도하고 있으며, 체르노빌 원전 지역에서도 격렬한 전투가 벌어지고 있습니다. 우크라이나는 이 지역의 통제권을 상실했으며, 방사능 위험에 대한 우려가 커지고 있습니다. 